# 作用域

在冯·诺依曼计算机体系结构的内存中，变量的属性可以视为一个六元组：（名字，地址，值，类型，生命期，作用域）。地址属性具有明显的冯·诺依曼体系结构的色彩，代表变量所关联的存储器地址。类型规定了变量的取值范围和可能的操作。生命期表示变量与某个存储区地址绑定的过程。根据生命期的不同，变量可以被分为四类：静态、栈动态、显式堆动态和隐式堆动态。作用域表征变量在语句中的可见范围，主要分为静态作用域和动态作用域两种。

作用域是所有编程语言都需要了解的基础概念之一，它是是为了解决开发过程中变量名冲突发展起来的概念。

## **Ruby** 变量

**Ruby** 中的变量主要有以下四种。

- 局部变量（a_var）: 可见范围取决于作用域。
- 全局变量（$a_var）：在程序的任何地方可见。
- 实例变量（@a_var）：在类实例访问及子类实例通过实例方法可见。
- 类变量（@@a_var）：当前类及其子类中可见，不能从其外的地方访问。

## 局部变量

> `Kernel#local_variables` 方法可以方便我们查看当前作用域中定义了哪些变量。

In [73]:
begin
  puts a_var              # NameError，需要先声明才可以访问
rescue NameError => e
  puts "#{e.message}\nlocals is #{local_variables}"
end

undefined local variable or method `a_var' for main:Object
locals is [:_i58, :_58, :e, :_i57, :_57, :_i56, :_56, :_i55, :_55, :_i54, :_54, :_i53, :_53, :_i52, :_52, :_i51, :_51, :_i50, :_50, :_i49, :_49, :_i48, :_48, :_i47, :_47, :_i46, :_46, :_i45, :_45, :_i44, :_44, :_i43, :_43, :_i42, :_42, :_i41, :_41, :_i40, :_40, :_i39, :_39, :_i38, :_38, :_i37, :_37, :_i36, :_36, :cbd, :_i35, :_35, :ccc, :_i34, :_34, :_i33, :_33, :_i32, :_32, :_i31, :_31, :_i30, :_30, :_i29, :_29, :_i28, :_28, :_i27, :_27, :_i26, :_26, :_i25, :_25, :_i24, :_24, :_i23, :_23, :_i22, :_22, :_i21, :_21, :_i20, :_20, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :_i13, :_13, :_i12, :_12, :binding1, :binding2, :_i11, :_11, :_i10, :_10, :_i9, :_9, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :_, :__, :___, :_i1, :_1, :main0, :obj, :_oh, :_ih, :title]


**Ruby** 语言是赋值即变量声明的语言，他没有与 **JavaScript** 语言的 `var x` 和 **Java** 的 `String str` 相当的变量声明。只能通过赋值语句来声明变量，但是却不需要该赋值语句真正被执行。

In [76]:
if false
  a_var = 100
end

puts a_var.nil?

true


### Scope Gates

每个 **Ruby** 作用域包含一组绑定，并且不同的作用域之间被 **Scope Gate** 分割开来。**Scope Gate** 的作用是关闭前一个作用域，同时打开一个新的作用域。**Ruby** 中使用 `class`，`module` 或 `def` 关键字来隔离作用域，也就是充当 **Scope Gate**。

先来看个例子：

In [18]:
x = 1
def change_x
  x = 2
end

change_x
x

1

In [28]:
%%file change_x.py

x = 1
def change_x():
  x = 2

change_x()
print("x = {0}".format(x))

Writing change_x.py


In [29]:
puts `python change_x.py`

x = 1



如你所料，`x` 的值并没有真正被修改。这和 **Python** 表现是一致的，但是其深层的原理却有比较大的差别。

在 **Python** 中，`change_x` 是一个闭包，它可以看到外部的 `x`，之所以没有修改成功，是因为 `change_x` 重新定义了本地变量 `x`，并把 `2` 赋值给了 `x`。

在 **Ruby** 中，`change_x` 则是打开了一个全新的作用域，它是无法访问到外部的变量 `x` 的。

In [77]:
%%file check_x.py

x = 1
def check_x():
  print("I can see x = {0} in check_x.".format(x))

check_x()

Writing check_x.py


In [78]:
puts `python check_x.py`

I can see x = 1 in check_x.



我们再来看一个多层 **Scope Gate** 的例子：

In [80]:
main0 = 'main0'

module MyModule
  module1 = 'module1'
  puts "enter MyModule locals: #{local_variables}"
  
  class MyClass
    class2 = 'class2'
    puts "enter MyClass locals: #{local_variables}"
    
    def my_method
      method_3 = 'method3'
      puts "my_method locals: #{local_variables}"
    end
    
    puts "exit MyClass locals: #{local_variables}"
  end
  
  puts "exit MyModule locals: #{local_variables}"
end
    
obj = MyModule::MyClass.new
obj.my_method
obj.my_method
  
puts "main locals: #{local_variables}"

enter MyModule locals: [:module1]
enter MyClass locals: [:class2]
exit MyClass locals: [:class2]
exit MyModule locals: [:module1]
my_method locals: [:method_3]
my_method locals: [:method_3]
main locals: [:_i64, :_64, :_i63, :_63, :_i62, :_62, :_i61, :_61, :_i60, :_60, :a_var, :_i59, :_59, :_i58, :_58, :e, :_i57, :_57, :_i56, :_56, :_i55, :_55, :_i54, :_54, :_i53, :_53, :_i52, :_52, :_i51, :_51, :_i50, :_50, :_i49, :_49, :_i48, :_48, :_i47, :_47, :_i46, :_46, :_i45, :_45, :_i44, :_44, :_i43, :_43, :_i42, :_42, :_i41, :_41, :_i40, :_40, :_i39, :_39, :_i38, :_38, :_i37, :_37, :_i36, :_36, :cbd, :_i35, :_35, :ccc, :_i34, :_34, :_i33, :_33, :_i32, :_32, :_i31, :_31, :_i30, :_30, :_i29, :_29, :_i28, :_28, :_i27, :_27, :_i26, :_26, :_i25, :_25, :_i24, :_24, :_i23, :_23, :_i22, :_22, :_i21, :_21, :_i20, :_20, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :_i13, :_13, :_i12, :_12, :binding1, :binding2, :_i11, :_11, :_i10, :_10, :_i9, :_9, :_i8, :_8, :_i7, :_7, :_

容易看出程序打开了五个独立的作用域

- 顶级作用域（`main0` 所在的作用域）
- 进入 `MyModule` 时创建的作用域
- 进入 `MyClass` 时创建的作用域
- 第一次调用 `my_method()` 方法时创建的一个作用域
- 第二次调用 `my_method()` 方法时创建的一个作用域

如何知道每次调用同一方法时会重新创建新的作用域而不是重用已有的，我们通过一个程序来验证。

In [81]:
class MyClass
  def my_method
    v = rand(1..100)
    puts "my_method locals: #{local_variables}"
    binding
  end 
end

obj = MyClass.new
binding1 = obj.my_method
binding2 = obj.my_method

puts "v in binding1 is #{binding1.local_variable_get(:v)}"
puts "v in binding2 is #{binding2.local_variable_get(:v)}"

binding1.local_variable_set(:v2, 1)

puts "binding1 variables is #{binding1.local_variables}"
puts "binding2 variables is #{binding2.local_variables}"

my_method locals: [:v]
my_method locals: [:v]
v in binding1 is 58
v in binding2 is 66
binding1 variables is [:v2, :v]
binding2 variables is [:v]


`class` / `module` 与 `def` 之间还有一点微妙的差别，在类和模块定义中的代码会被立即执行，相反，方法定义中的代码只有在方法被调用的时候才会被执行。

### 顶级上下文（Top Level Context）

当开始运行 **Ruby** 程序时，**Ruby** 虚拟机会创建一个名为 `main` 的对象作为当前对象，这个对象被称为顶级上下文，这个名字的由来是因为这时处在调用栈的顶层，这时要么还没调用任何方法，要么调用的所有方法都已经返回了。

In [82]:
puts self
puts self.class
puts local_variables

main
Object
[:_i66, :_66, :_i65, :_65, :_i64, :_64, :_i63, :_63, :_i62, :_62, :_i61, :_61, :_i60, :_60, :a_var, :_i59, :_59, :_i58, :_58, :e, :_i57, :_57, :_i56, :_56, :_i55, :_55, :_i54, :_54, :_i53, :_53, :_i52, :_52, :_i51, :_51, :_i50, :_50, :_i49, :_49, :_i48, :_48, :_i47, :_47, :_i46, :_46, :_i45, :_45, :_i44, :_44, :_i43, :_43, :_i42, :_42, :_i41, :_41, :_i40, :_40, :_i39, :_39, :_i38, :_38, :_i37, :_37, :_i36, :_36, :cbd, :_i35, :_35, :ccc, :_i34, :_34, :_i33, :_33, :_i32, :_32, :_i31, :_31, :_i30, :_30, :_i29, :_29, :_i28, :_28, :_i27, :_27, :_i26, :_26, :_i25, :_25, :_i24, :_24, :_i23, :_23, :_i22, :_22, :_i21, :_21, :_i20, :_20, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :_i13, :_13, :_i12, :_12, :binding1, :binding2, :_i11, :_11, :_i10, :_10, :_i9, :_9, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :_, :__, :___, :_i1, :_1, :main0, :obj, :_oh, :_ih, :title]


我们现在运行代码的作用域就是顶级实例变量 `main` 所在的作用域。

### 词法作用域

词法作用域(lexical scope)也叫静态作用域(static scope)。它其实是指作用域在词法解析阶段既确定了，不会改变。在 每个 **Scope Gate** 内部，**Ruby** 局部变量遵循词法作用域规则。

In [94]:
module CleanRoom2
  
  outer1 = lambda do
    outer1_v = 0
    inner1 = lambda do
      inter1_v = 0
      local_variables
    end
  end

  outer2 = lambda do
    outer2_v = 0
    inner2 = lambda do 
      inner2_v = 0
      outer1.call.call
    end
  end

  outer2.call.call
end

[:inter1_v, :outer1_v, :inner1, :outer1, :outer2]

可以看到 `inner1` 是在 `outer1` 中定义，调用是在 `inner2` 块中，可以看到在 `inner1` 中的可见的变量有定义在 `CleanRoom` 中的所有变量`[:outer1, :outer2]`，定义在 `outer1` 中的 `[:outer1_v, :inner1]` 和定义在 `inner1` 中的 `[:inner1_v]`。尽管执行是在 `outer2` 和 `inner2` 的内部，但是 `inner1` 并不能访问它们的变量定义。

通俗地说，代码块会在定义时获取周围的绑定，也可以在代码块中定义额外的绑定，但这些绑定会在块结束时消失。

In [97]:
module CleanRoom
  
  outer1 = lambda do
    outer1_v = 0
    inner1 = lambda do
      inter1_v = 0
      puts "inner1 locals: #{local_variables}"
    end
    puts "outer1 locals: #{local_variables}"
    inner1
  end
  
  puts "CleanRoom locals: #{local_variables}"

  outer2 = lambda do
    outer2_v = 0
    inner2 = lambda do 
      inner2_v = 0
      outer1.call.call
    end
  end

  outer2.call.call
end

CleanRoom locals: [:outer1, :outer2]
outer1 locals: [:outer1_v, :inner1, :outer1, :outer2]
inner1 locals: [:inter1_v, :outer1_v, :inner1, :outer1, :outer2]


在代码块中，声明变量前，总是先往上查找上层环境是不是已经有该变量，如有，则直接重用，若没有则在当前块中声明该变量。

In [118]:
def foo()
  x = 'old'
  lambda do 
    x = 'new' 
  end.call
  x
end

foo()

"new"

### 动态作用域

与词法作用域中作用域是源代码级别上的一块完整独立的范围不同，在动态作用域中，作用域则是进入该作用域开始直至离开这一时间轴上的完整独立的范围。与此相同的特征也体现在其他好多地方。比如，在某处理进行期间，一时改变某变量的值随后将原值返回的代码编写方式就相当于创建了自己专属的动态作用域。又如，异常处理与动态作用域也很相似，函数抛出异常时的处理方式受到调用函数的 try/ catch语句的影响。

词法作用域：讨论代码的组织结构上的抽象，讨论的是“圈地”的问题（形式上的规范）。
动态作用域：变量的可见性，完成对信息的隐藏，也就是处理“割据”问题（实际的占有）。

现在很少会使用动态作用域，因为动态作用域中被改写的值会影响到被调用的函数，因此在引用变量时它是什么样的值，不看调用方的代码是无从得知的。如果只是为了获取调用处的值，完全可以通过参数传递来解决。动态作用域和词法作用域实现上的差别，可以参考[我之前的文章](https://github.com/jameszhan/blogs/blob/master/_posts/2015-04-01-build-interpret-from-zero.md)。  

由于 **Ruby** 不支持动态作用域，下面的例子，我们使用 **Perl** 语言来演示，它同时支持词法作用域和动态作用域。

In [119]:
%%file dynamic_scope.pl
sub invoker {
    local $x = 'invoker';
    &func();
}

$x = 'global';
sub func {
    print "$x\n";
}
invoker()

Writing dynamic_scope.pl


In [120]:
`perl dynamic_scope.pl`

"invoker\n"

## 全局变量

In [121]:
global_variables

[:$_, :$~, :$;, :$-F, :$@, :$!, :$SAFE, :$&, :$`, :$', :$+, :$=, :$KCODE, :$-K, :$,, :$/, :$-0, :$\, :$stdin, :$stdout, :$stderr, :$>, :$<, :$., :$FILENAME, :$-i, :$*, :$:, :$-I, :$LOAD_PATH, :$", :$LOADED_FEATURES, :$?, :$$, :$VERBOSE, :$-v, :$-w, :$-W, :$DEBUG, :$-d, :$0, :$PROGRAM_NAME, :$-p, :$-l, :$-a, :$fileutils_rb_have_lchmod, :$fileutils_rb_have_lchown, :$my_var]

**Ruby** 中只要变量名称以 `$` 开头，这个变量就是全局变量。全局变量可以再任何作用域中访问。

In [122]:
def scope1
  $my_var = 100
end

def scope2
  puts "$my_var is #{$my_var}"
end

scope2
scope1
scope2

global_variables

$my_var is 100
$my_var is 100


[:$_, :$~, :$;, :$-F, :$@, :$!, :$SAFE, :$&, :$`, :$', :$+, :$=, :$KCODE, :$-K, :$,, :$/, :$-0, :$\, :$stdin, :$stdout, :$stderr, :$>, :$<, :$., :$FILENAME, :$-i, :$*, :$:, :$-I, :$LOAD_PATH, :$", :$LOADED_FEATURES, :$?, :$$, :$VERBOSE, :$-v, :$-w, :$-W, :$DEBUG, :$-d, :$0, :$PROGRAM_NAME, :$-p, :$-l, :$-a, :$fileutils_rb_have_lchmod, :$fileutils_rb_have_lchown, :$my_var]

全局变量的问题在于系统中的任何部分都可以修改它们，我们几乎没法追踪谁把他们改成了什么。因此，基本的原则是：如非必要，尽可能少使用全局变量。

## 实例变量

In [7]:
class MyClass
  
end

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class`] <br> `obj` | `var = 1`<br>`access_class`  |

In [8]:
obj = MyClass()

查询 `self.var` 时，由于 `obj` 不存在 `var`，所以跳到 MyClass 中：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [9]:
obj.access_class_c()

class var: 1


查询 `var` 直接跳到 `global` 作用域：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [10]:
obj.access_global_c()

global var: 0


修改类中的 `MyClass.var`：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  |

In [11]:
obj.write_class_c()

class var: 2


修改实例中的 `var` 时，会直接在 `obj` 域中创建一个：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  | `var = 3`

In [12]:
obj.write_instance_c()

instance var: 3


In [13]:
MyClass.var

2

`MyClass` 中的 `var` 并没有改变。

In [14]:
def outer():
    a = 1
    def inner():
        print "a =", a
    inner()
    
outer()

a = 1


如果里面的函数没有找到变量，那么会向外一层寻找变量，如果再找不到，则到 `global` 作用域。

返回的是函数的情况：

In [15]:
def outer():
    a = 1
    def inner():
        return a
    return inner
    
func = outer()

print 'a (1):', func()

a (1): 1


func() 函数中调用的 `a` 要从它定义的地方开始寻找，而不是在 `func` 所在的作用域寻找。